In [34]:
import numpy as np
import spatial
import importlib
importlib.reload(spatial)
from polynomial import Polymonial, multiXPoly, addXPoly

def calculate_Lambda(lambdas, alpha):
    # Begin algorithm 1
    R = len(lambdas)
    Ss = []
    for k in range(0, R):
        P = np.exp(-1j * alpha * lambdas[k])
        Vs = []
        for l in range(0, R):
            if l != k: 
                P = P / (lambdas[k] - lambdas[l])
                Vs.append(Polymonial([-lambdas[l], 1]))
        V = multiXPoly(Vs)
        Ss.append(V.multiX(P))
    S = addXPoly(Ss)
    return S
def calculate_M(lambdas, alpha):
    # This return Lambda_0, Lambda_1, ...
    Lambdas = calculate_Lambda(lambdas, alpha).coeff
    M = np.zeros([len(Lambdas), len(Lambdas)], dtype = np.complex128)
    for i in range(0, len(Lambdas)):
        for j in range(0, len(Lambdas)):
            M[i, j] = np.conjugate(Lambdas[i]) * Lambdas[j]
    return M

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, np.conjugate(a.T), rtol=rtol, atol=atol)
def calculate_distance_matrix(A, B):
    d = 0
    for i in range(0, A.shape[0]):
        for j in range(0, A.shape[1]):
            d += np.abs(A[i, j] - B[i, j])**2
    return np.sqrt(d)
    
# alpha = -np.pi / 2
# lambdas = [-1, 0, 1]
# S = calculate_Lambda(lambdas, alpha)
# print(S.coeff)

In [35]:

def calculate_pseudo_c(d3, c):
    return d3*(calculate_M(lambdas, c) - calculate_M(lambdas, -c))
def loss(d3, c):
    return calculate_distance_matrix(calculate_pseudo_c(d3, c), C)
alpha = np.pi / 4
beta = np.pi / 2
c = 3 * np.pi / 2
# lambdas = [-1, 0, 1]
lambdas = [-3/2, -1/2, 1/2, 3/2]
Lambdas = calculate_Lambda(lambdas, alpha).coeff
delta_Malpha = calculate_M(lambdas, alpha) - calculate_M(lambdas, -alpha)
delta_Mbeta = calculate_M(lambdas, beta) - calculate_M(lambdas, -beta)
delta_Mc = calculate_M(lambdas, c) - calculate_M(lambdas, -c)

d1 = 1/2
d2 = (-np.sqrt(2) + 1) / 4


# Create target matrix
X = d1*delta_Malpha + d2*delta_Mbeta
C = -X.copy()
C[0, 1] = 0
C[1, 0] = 0 




searcher = spatial.SpatialSearch(loss_func = loss, radius = 1, center = (1, np.pi))
solution = searcher.fit(learning_rate = 2, threshold=10**(-4), limit_step = 1000)


In [40]:
np.round(solution[0]*calculate_M(lambdas, c) - calculate_M(lambdas, -c), 3)

array([[ 2.98956825e+13+0.00000000e+00j, -0.00000000e+00+5.18191829e+13j,
        -2.39165460e+13-1.00000000e-03j,  0.00000000e+00-1.59443640e+13j],
       [-0.00000000e+00-5.18191829e+13j,  8.98199171e+13+0.00000000e+00j,
        -1.00000000e-03+4.14553463e+13j, -2.76368976e+13-1.00000000e-03j],
       [-2.39165460e+13+1.00000000e-03j, -1.00000000e-03-4.14553463e+13j,
         1.91332368e+13+0.00000000e+00j,  0.00000000e+00+1.27554912e+13j],
       [ 0.00000000e+00+1.59443640e+13j, -2.76368976e+13+1.00000000e-03j,
         0.00000000e+00-1.27554912e+13j,  8.50366079e+12+0.00000000e+00j]])

In [41]:
calculate_pseudo_c(solution[0], solution[1])

array([[0.+0.00000000e+00j, 0.-1.64020200e-02j, 0.+0.00000000e+00j,
        0.+9.37258287e-03j],
       [0.+1.64020200e-02j, 0.+0.00000000e+00j, 0.-2.18693600e-02j,
        0.-1.41501211e-18j],
       [0.+0.00000000e+00j, 0.+2.18693600e-02j, 0.+0.00000000e+00j,
        0.-1.24967772e-02j],
       [0.-9.37258287e-03j, 0.+1.41501211e-18j, 0.+1.24967772e-02j,
        0.+0.00000000e+00j]])

In [20]:
X + C

array([[0.+0.j        , 0.-0.49694903j, 0.+0.j        , 0.+0.j        ],
       [0.+0.49694903j, 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.+0.j        ]])

In [79]:
alpha = np.pi / 4
beta = 3*np.pi / 4
lambdas = [-1, 0, 1]

delta_Malpha = calculate_M(lambdas, alpha) - calculate_M(lambdas, -alpha)
delta_Mbeta = calculate_M(lambdas, beta) - calculate_M(lambdas, -beta)

d1 = (np.sqrt(2) + 1)/(4*np.sqrt(2))
d2 = (-np.sqrt(2) + 1)/(4*np.sqrt(2))

print(d1*delta_Malpha + d2*delta_Mbeta)

[[0.+0.00000000e+00j 0.-5.00000000e-01j 0.+0.00000000e+00j]
 [0.+5.00000000e-01j 0.+0.00000000e+00j 0.+5.55111512e-17j]
 [0.+0.00000000e+00j 0.-5.55111512e-17j 0.+0.00000000e+00j]]
